In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk.data
import logging
import numpy as np  # Make sure that numpy is imported
from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from autocorrect import spell
import re
from textblob import TextBlob


# from KaggleWord2VecUtility import KaggleWord2VecUtility

C:\Users\Lenovo\Anaconda3\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [10]:
train = pd.read_csv("train.csv")
b=[]
for x in train["Description"][0:2]:
    b.append(TextBlob(str(x)).correct())
# train.Description = b
# train.head()
b

[TextBlob("The room was kind of clean but had a VERY strong smell of dogs. Generally below average but ok for a overnight stay if you're not too fussy. Would consider staying again if the price was right. Breakfast was free and just about better than nothing."),
 TextBlob("I stayed at the Grown Lazy April -- - April --, ----. The staff was friendly and attentive. The elevators are tiny (about -' by -'). The food in the restaurant was delicious but prices a little on the high side. Of course this is Washington of. There is no pool and little for children to do. By room on the fifth floor had two comfortable beds and plenty of space for one person. The of is a little small by today standards with a limited number of channels. There was a small bit of mold in the bathtubs area that could have been removed with a little breach. It appeared the carpets were not scummed every day. I reported a light bulb was burned out. It was never replaced. Ice machines are on the odd numbered floors, but 

In [ ]:
test = pd.read_csv("test.csv")
b=[]
for x in test["Description"]:
    b.append(TextBlob(str(x)).correct())
test.Description = b
test.head()

In [ ]:
test["Is_Response"]  = np.nan
alldata = pd.concat([train,test]).reset_index(drop = True)
print (alldata.shape)
alldata.head()

In [15]:
def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words and autocorrect the misspells
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))   

In [23]:
def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
   
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [13]:
import nltk.data
nltk.download()   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [24]:
sentences = []  # Initialize an empty list of sentences

print ("Parsing sentences from alldata")
for review in alldata["Description"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from alldata


In [29]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print( "Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2017-10-03 15:51:38,898 : INFO : collecting all words and their counts
2017-10-03 15:51:38,902 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-10-03 15:51:38,977 : INFO : PROGRESS: at sentence #10000, processed 158651 words, keeping 8456 word types
2017-10-03 15:51:39,049 : INFO : PROGRESS: at sentence #20000, processed 315617 words, keeping 11663 word types


Training model...


2017-10-03 15:51:39,104 : INFO : PROGRESS: at sentence #30000, processed 474180 words, keeping 14218 word types
2017-10-03 15:51:39,167 : INFO : PROGRESS: at sentence #40000, processed 630235 words, keeping 16294 word types
2017-10-03 15:51:39,256 : INFO : PROGRESS: at sentence #50000, processed 789819 words, keeping 18009 word types
2017-10-03 15:51:39,341 : INFO : PROGRESS: at sentence #60000, processed 947721 words, keeping 19603 word types
2017-10-03 15:51:39,390 : INFO : PROGRESS: at sentence #70000, processed 1109181 words, keeping 21078 word types
2017-10-03 15:51:39,482 : INFO : PROGRESS: at sentence #80000, processed 1267372 words, keeping 22376 word types
2017-10-03 15:51:39,549 : INFO : PROGRESS: at sentence #90000, processed 1424951 words, keeping 23582 word types
2017-10-03 15:51:39,637 : INFO : PROGRESS: at sentence #100000, processed 1584578 words, keeping 24712 word types
2017-10-03 15:51:39,700 : INFO : PROGRESS: at sentence #110000, processed 1740689 words, keeping 25

2017-10-03 15:51:44,957 : INFO : resetting layer weights
2017-10-03 15:51:45,245 : INFO : training model with 4 workers on 6902 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2017-10-03 15:51:46,309 : INFO : PROGRESS: at 0.52% examples, 191755 words/s, in_qsize 7, out_qsize 0
2017-10-03 15:51:47,337 : INFO : PROGRESS: at 1.42% examples, 260814 words/s, in_qsize 7, out_qsize 0
2017-10-03 15:51:48,359 : INFO : PROGRESS: at 2.43% examples, 297878 words/s, in_qsize 7, out_qsize 0
2017-10-03 15:51:49,399 : INFO : PROGRESS: at 3.41% examples, 312287 words/s, in_qsize 7, out_qsize 0
2017-10-03 15:51:50,392 : INFO : PROGRESS: at 4.39% examples, 322421 words/s, in_qsize 7, out_qsize 0
2017-10-03 15:51:51,426 : INFO : PROGRESS: at 5.41% examples, 330054 words/s, in_qsize 7, out_qsize 0
2017-10-03 15:51:52,447 : INFO : PROGRESS: at 6.20% examples, 324534 words/s, in_qsize 7, out_qsize 1
2017-10-03 15:51:53,489 : INFO : PROGRESS: at 6.86% examples, 313499 words/s, i

2017-10-03 16:02:20,751 : INFO : PROGRESS: at 41.45% examples, 24296 words/s, in_qsize 0, out_qsize 11
2017-10-03 16:02:22,080 : INFO : PROGRESS: at 41.47% examples, 24257 words/s, in_qsize 0, out_qsize 11
2017-10-03 16:02:24,410 : INFO : PROGRESS: at 41.49% examples, 24179 words/s, in_qsize 0, out_qsize 11
2017-10-03 16:02:25,876 : INFO : PROGRESS: at 41.51% examples, 24134 words/s, in_qsize 0, out_qsize 11
2017-10-03 16:02:26,941 : INFO : PROGRESS: at 41.53% examples, 24105 words/s, in_qsize 1, out_qsize 11
2017-10-03 16:02:30,395 : INFO : PROGRESS: at 41.54% examples, 23987 words/s, in_qsize 0, out_qsize 11
2017-10-03 16:02:33,802 : INFO : PROGRESS: at 41.56% examples, 23871 words/s, in_qsize 2, out_qsize 11
2017-10-03 16:02:34,813 : INFO : PROGRESS: at 41.58% examples, 23845 words/s, in_qsize 3, out_qsize 11
2017-10-03 16:02:36,981 : INFO : PROGRESS: at 41.60% examples, 23776 words/s, in_qsize 4, out_qsize 11
2017-10-03 16:02:39,405 : INFO : PROGRESS: at 41.62% examples, 23698 word

2017-10-03 16:04:25,708 : INFO : PROGRESS: at 46.72% examples, 22893 words/s, in_qsize 0, out_qsize 2
2017-10-03 16:04:27,550 : INFO : PROGRESS: at 46.84% examples, 22892 words/s, in_qsize 0, out_qsize 0
2017-10-03 16:04:28,996 : INFO : PROGRESS: at 46.85% examples, 22858 words/s, in_qsize 0, out_qsize 1
2017-10-03 16:04:30,342 : INFO : PROGRESS: at 46.95% examples, 22863 words/s, in_qsize 0, out_qsize 0
2017-10-03 16:04:31,564 : INFO : PROGRESS: at 47.04% examples, 22871 words/s, in_qsize 0, out_qsize 0
2017-10-03 16:04:32,939 : INFO : PROGRESS: at 47.12% examples, 22866 words/s, in_qsize 1, out_qsize 0
2017-10-03 16:04:34,174 : INFO : PROGRESS: at 47.21% examples, 22874 words/s, in_qsize 0, out_qsize 0
2017-10-03 16:04:35,347 : INFO : PROGRESS: at 47.23% examples, 22848 words/s, in_qsize 0, out_qsize 1
2017-10-03 16:04:36,633 : INFO : PROGRESS: at 47.32% examples, 22854 words/s, in_qsize 0, out_qsize 0
2017-10-03 16:04:37,832 : INFO : PROGRESS: at 47.40% examples, 22855 words/s, in_q

2017-10-03 16:06:08,314 : INFO : PROGRESS: at 58.76% examples, 25360 words/s, in_qsize 8, out_qsize 1
2017-10-03 16:06:09,319 : INFO : PROGRESS: at 59.74% examples, 25752 words/s, in_qsize 7, out_qsize 0
2017-10-03 16:06:10,327 : INFO : PROGRESS: at 60.74% examples, 26152 words/s, in_qsize 7, out_qsize 0
2017-10-03 16:06:11,344 : INFO : PROGRESS: at 61.66% examples, 26518 words/s, in_qsize 7, out_qsize 0
2017-10-03 16:06:12,348 : INFO : PROGRESS: at 62.71% examples, 26941 words/s, in_qsize 7, out_qsize 0
2017-10-03 16:06:13,378 : INFO : PROGRESS: at 63.82% examples, 27385 words/s, in_qsize 7, out_qsize 0
2017-10-03 16:06:14,407 : INFO : PROGRESS: at 64.69% examples, 27725 words/s, in_qsize 7, out_qsize 0
2017-10-03 16:06:15,405 : INFO : PROGRESS: at 65.67% examples, 28112 words/s, in_qsize 7, out_qsize 0
2017-10-03 16:06:16,423 : INFO : PROGRESS: at 66.60% examples, 28482 words/s, in_qsize 6, out_qsize 1
2017-10-03 16:06:17,421 : INFO : PROGRESS: at 67.52% examples, 28836 words/s, in_q

In [35]:



# ****** Define functions to create average word vectors
#

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    #
    # Index2word is a list that contains the names of the words in
    # the model's vocabulary. Convert it to a set, for speed
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    #
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate
    # the average feature vector for each one and return a 2D numpy array
    #
    # Initialize a counter
    counter = 0.
    #
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    #
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print ("Review %d of %d" % (counter, len(reviews)))
       #
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[int(counter)] = makeFeatureVec(review, model, \
           num_features)
       #
       # Increment the counter
       counter = counter + 1.
    return reviewFeatureVecs


def getCleanReviews(reviews):
    clean_reviews = []
    for review in reviews:
        clean_reviews.append( review_to_wordlist( review, remove_stopwords=True ))
    return clean_reviews





    

   


    # Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')





   

    
    # ****** Create average vectors for the training and test sets
    #
print ("Creating average feature vecs for training reviews")

trainDataVecs = getAvgFeatureVecs( getCleanReviews(train.Description), model, num_features )

print ("Creating average feature vecs for test reviews")

testDataVecs = getAvgFeatureVecs( getCleanReviews(test.Description), model, num_features )


  

   

Creating average feature vecs for training reviews
Review 0 of 38932
Review 1000 of 38932
Review 2000 of 38932
Review 3000 of 38932
Review 4000 of 38932
Review 5000 of 38932
Review 6000 of 38932
Review 7000 of 38932
Review 8000 of 38932
Review 9000 of 38932
Review 10000 of 38932
Review 11000 of 38932
Review 12000 of 38932
Review 13000 of 38932
Review 14000 of 38932
Review 15000 of 38932
Review 16000 of 38932
Review 17000 of 38932
Review 18000 of 38932
Review 19000 of 38932
Review 20000 of 38932
Review 21000 of 38932
Review 22000 of 38932
Review 23000 of 38932
Review 24000 of 38932
Review 25000 of 38932
Review 26000 of 38932
Review 27000 of 38932
Review 28000 of 38932
Review 29000 of 38932
Review 30000 of 38932
Review 31000 of 38932
Review 32000 of 38932
Review 33000 of 38932
Review 34000 of 38932
Review 35000 of 38932
Review 36000 of 38932
Review 37000 of 38932
Review 38000 of 38932
Creating average feature vecs for test reviews
Review 0 of 29404
Review 1000 of 29404
Review 2000 of 294

KeyError: 'id'

In [54]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
col = ["Browser_Used", "Device_Used"]
for x in col:
    test[x] = le.fit_transform(test[x])
    train[x] = le.fit_transform(train[x])

In [56]:
# trainDataVecs = trainDataVecs[0:,0:-8] 
trainDataVecs1 = np.concatenate((trainDataVecs,(train.ix[0:,2:4]).as_matrix()),axis = 1)
print (trainDataVecs1)
testDataVecs1 = np.concatenate((testDataVecs,(test.ix[0:,2:4]).as_matrix()),axis = 1)
print (testDataVecs1)


[[-9.143814168055542e-06 9.365867299493402e-06 -0.027765708044171333 ...,
  -0.0007005080115050077 1 1]
 [-0.005732249468564987 -0.0006221746443770826 0.0036955680698156357 ...,
  0.015216168016195297 5 1]
 [-0.008366284891963005 0.006637528073042631 -0.024731799960136414 ...,
  0.0055737621150910854 7 2]
 ..., 
 [-0.0030185196083039045 -0.0034438427537679672 0.01082073524594307 ...,
  -0.014367852360010147 2 1]
 [-0.004637942649424076 0.0023499864619225264 0.015916388481855392 ...,
  0.017617862671613693 0 0]
 [0.030391767621040344 0.0041166082955896854 -0.02241602912545204 ...,
  0.00910718459635973 7 0]]


C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.


[[  1.13193709e-02  -1.35587454e-02   2.11760290e-02 ...,  -1.25811156e-02
    2.00000000e+00   1.00000000e+00]
 [  1.56677514e-02   7.91391917e-03  -1.22238742e-02 ...,  -2.63791960e-02
    6.00000000e+00   0.00000000e+00]
 [  2.02242117e-02  -6.75310730e-04   1.44710531e-02 ...,  -6.95725530e-03
    4.00000000e+00   2.00000000e+00]
 ..., 
 [ -9.14212083e-04  -1.29836015e-02  -2.06453027e-03 ...,   3.84921557e-03
    1.00000000e+00   0.00000000e+00]
 [  2.43215617e-02  -1.94434319e-02   2.50802543e-02 ...,   1.62488420e-03
    6.00000000e+00   0.00000000e+00]
 [ -5.71358344e-03  -6.55804062e-03  -1.73102636e-02 ...,   3.38215032e-03
    8.00000000e+00   1.00000000e+00]]


In [57]:
  # ****** Fit a random forest to the training set, then make predictions
    #
    # Fit a random forest to the training data, using 100 trees
forest = RandomForestClassifier( n_estimators = 100 )

print ("Fitting a random forest to labeled training data...")
forest = forest.fit( trainDataVecs1, train["Is_Response"] )

    # Test & extract results
result = forest.predict( testDataVecs1 )

Fitting a random forest to labeled training data...


In [58]:
result

array(['not happy', 'happy', 'happy', ..., 'happy', 'happy', 'happy'], dtype=object)

In [59]:
 # Write the test results
output = pd.DataFrame( {"User_ID":test["User_ID"], "Is_Response":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )
print ("Wrote Word2Vec_AverageVectors.csv")##gave me 63% accuracy

Wrote Word2Vec_AverageVectors.csv


In [ ]:
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
import time
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import numpy as np
import os
from KaggleWord2VecUtility import KaggleWord2VecUtility


# Define a function to create bags of centroids
#
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids


if __name__ == '__main__':

    model = Word2Vec.load("300features_40minwords_10context")


    # ****** Run k-means on the word vectors and print a few clusters
    #

    start = time.time() # Start time

    # Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
    # average of 5 words per cluster
    word_vectors = model.wv.syn0
    num_clusters = word_vectors.shape[0] / 5

    # Initalize a k-means object and use it to extract centroids
    print "Running K means"
    kmeans_clustering = KMeans( n_clusters = num_clusters )
    idx = kmeans_clustering.fit_predict( word_vectors )

    # Get the end time and print how long the process took
    end = time.time()
    elapsed = end - start
    print "Time taken for K Means clustering: ", elapsed, "seconds."


    # Create a Word / Index dictionary, mapping each vocabulary word to
    # a cluster number
    word_centroid_map = dict(zip( model.wv.index2word, idx ))

    # Print the first ten clusters
    for cluster in xrange(0,10):
        #
        # Print the cluster number
        print "\nCluster %d" % cluster
        #
        # Find all of the words for that cluster number, and print them out
        words = []
        for i in xrange(0,len(word_centroid_map.values())):
            if( word_centroid_map.values()[i] == cluster ):
                words.append(word_centroid_map.keys()[i])
        print words




    # Create clean_train_reviews and clean_test_reviews as we did before
    #

    # Read data from files
    train = pd.read_csv( os.path.join(os.path.dirname(__file__), 'data', 'labeledTrainData.tsv'), header=0, delimiter="\t", quoting=3 )
    test = pd.read_csv(os.path.join(os.path.dirname(__file__), 'data', 'testData.tsv'), header=0, delimiter="\t", quoting=3 )


    print "Cleaning training reviews"
    clean_train_reviews = []
    for review in train["review"]:
        clean_train_reviews.append( KaggleWord2VecUtility.review_to_wordlist( review, \
            remove_stopwords=True ))

    print "Cleaning test reviews"
    clean_test_reviews = []
    for review in test["review"]:
        clean_test_reviews.append( KaggleWord2VecUtility.review_to_wordlist( review, \
            remove_stopwords=True ))


    # ****** Create bags of centroids
    #
    # Pre-allocate an array for the training set bags of centroids (for speed)
    train_centroids = np.zeros( (train["review"].size, num_clusters), \
        dtype="float32" )

    # Transform the training set reviews into bags of centroids
    counter = 0
    for review in clean_train_reviews:
        train_centroids[counter] = create_bag_of_centroids( review, \
            word_centroid_map )
        counter += 1

    # Repeat for test reviews
    test_centroids = np.zeros(( test["review"].size, num_clusters), \
        dtype="float32" )

    counter = 0
    for review in clean_test_reviews:
        test_centroids[counter] = create_bag_of_centroids( review, \
            word_centroid_map )
        counter += 1


    # ****** Fit a random forest and extract predictions
    #
    forest = RandomForestClassifier(n_estimators = 100)

    # Fitting the forest may take a few minutes
    print "Fitting a random forest to labeled training data..."
    forest = forest.fit(train_centroids,train["sentiment"])
    result = forest.predict(test_centroids)

    # Write the test results
    output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
    output.to_csv("BagOfCentroids.csv", index=False, quoting=3)
    print "Wrote BagOfCentroids.csv"

## doc2vec approach without removing stopwords and tunning the model and light gbm parameters

In [ ]:
from nltk.stem import WordNetLemmatizer
Lemma = WordNetLemmatizer()
def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    
   
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    #
#     #4 
    words = " ".join([Lemma.lemmatize(x) for x in words])
             
#     # 
#     # 5. Remove stop words and autocorrect the misspells
#     meaningful_words = [spell(w) for w in words if not w in stops]   
#     #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return(words)   

In [ ]:
from nltk.stem import WordNetLemmatizer
Lemma = WordNetLemmatizer()
def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
   
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review)
    
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    words = [Lemma.lemmatize(x) for x in words]
    #
    # 5. Return a list of words
    return(words)

In [65]:
import nltk.data
# nltk.download()   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [ ]:
sentences = []  # Initialize an empty list of sentences

print ("Parsing sentences from alldata")
for review in alldata["Description"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from alldata


In [21]:
sentences

[['the',
  'room',
  'was',
  'kind',
  'of',
  'clean',
  'but',
  'had',
  'a',
  'very',
  'strong',
  'smell',
  'of',
  'dogs'],
 ['generally',
  'below',
  'average',
  'but',
  'ok',
  'for',
  'a',
  'overnight',
  'stay',
  'if',
  'you',
  're',
  'not',
  'too',
  'fussy'],
 ['would',
  'consider',
  'staying',
  'again',
  'if',
  'the',
  'price',
  'was',
  'right'],
 ['breakfast',
  'was',
  'free',
  'and',
  'just',
  'about',
  'better',
  'than',
  'nothing'],
 ['i', 'stayed', 'at', 'the', 'crown', 'plaza', 'april', 'april'],
 ['the', 'staff', 'was', 'friendly', 'and', 'attentive'],
 ['the', 'elevators', 'are', 'tiny', 'about', 'by'],
 ['the',
  'food',
  'in',
  'the',
  'restaurant',
  'was',
  'delicious',
  'but',
  'priced',
  'a',
  'little',
  'on',
  'the',
  'high',
  'side'],
 ['of', 'course', 'this', 'is', 'washington', 'dc'],
 ['there', 'is', 'no', 'pool', 'and', 'little', 'for', 'children', 'to', 'do'],
 ['my',
  'room',
  'on',
  'the',
  'fifth',
  'fl

In [ ]:
sentences = []
word_list =[review_to_wordlist(x) for x in alldata["Description"]]
label_list = [str(label %s)%(i for i in range(68336))]
for s,v in zip(word_list,label_list):
    sentence.append(LabeledSentence(words=s, labels=v))
            


In [15]:
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
model = doc2vec.Doc2Vec(min_count=40, window=20, size=1000,
 sample=1e-4,workers=4)
model

In [19]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 1000    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 20          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import doc2vec
model = doc2vec.Doc2Vec(sentences,min_count=40, window=20, size=1000, sample=1e-4,workers=4)
model
# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "Doc2Vec_1000features_40minwords_20context(with stopwords)"
model.save(model_name)

2017-10-04 13:34:27,449 : INFO : collecting all words and their counts


AttributeError: 'list' object has no attribute 'words'

In [ ]:
def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    #
    # Index2word is a list that contains the names of the words in
    # the model's vocabulary. Convert it to a set, for speed
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    #
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate
    # the average feature vector for each one and return a 2D numpy array
    #
    # Initialize a counter
    counter = 0.
    #
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    #
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print ("Review %d of %d" % (counter, len(reviews)))
       #
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[int(counter)] = makeFeatureVec(review, model, \
           num_features)
       #
       # Increment the counter
       counter = counter + 1.
    return reviewFeatureVecs


def getCleanReviews(reviews):
    clean_reviews = []
    for review in reviews:
        clean_reviews.append( review_to_wordlist( review, remove_stopwords=True ))
    return clean_reviews





    

   


    # Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')





   

    
    # ****** Create average vectors for the training and test sets
    #
print ("Creating average feature vecs for training reviews")

trainDataVecs = getAvgFeatureVecs( getCleanReviews(train.Description), model, num_features )

print ("Creating average feature vecs for test reviews")

testDataVecs = getAvgFeatureVecs( getCleanReviews(test.Description), model, num_features )


  

   

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
col = ["Browser_Used", "Device_Used"]
for x in col:
    test[x] = le.fit_transform(test[x])
    train[x] = le.fit_transform(train[x])

In [ ]:
# trainDataVecs = trainDataVecs[0:,0:-8] 
trainDataVecs1 = np.concatenate((trainDataVecs,(train.ix[0:,2:4]).as_matrix()),axis = 1)
print (trainDataVecs1)
testDataVecs1 = np.concatenate((testDataVecs,(test.ix[0:,2:4]).as_matrix()),axis = 1)
print (testDataVecs1)


In [ ]:
  # ****** Fit a random forest to the training set, then make predictions
    #
    # Fit a random forest to the training data, using 100 trees
forest = RandomForestClassifier( n_estimators = 100 )

print ("Fitting a random forest to labeled training data...")
forest = forest.fit( trainDataVecs1, train["Is_Response"] )

    # Test & extract results
result = forest.predict( testDataVecs1 )

In [ ]:
output = pd.DataFrame( {"User_ID":test["User_ID"], "Is_Response":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )
print ("Wrote Word2Vec_AverageVectors.csv")##gave me 63% accuracy